In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import ServerlessSpec,Pinecone as pt
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os
from uuid import uuid4
from langchain.embeddings import OpenAIEmbeddings
import openai
from langchain.chat_models import ChatOpenAI

/home/ali/Desktop/My Projects/Chatbot-Development-with-Python-for-AI-ML/ApplabQatar/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

True

In [8]:
PINCECONE_API_KEYS = os.getenv('PINCECONE_API_KEYS')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("/home/ali/Desktop/Final Year Project/backend/python/dataTesting/PDF FIles for Inserting in Pincone/")

In [ ]:
extracted_data

In [ ]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

In [ ]:
text_chunks

In [ ]:
embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [9]:
pc = pt(
    api_key=PINCECONE_API_KEYS,
)

In [10]:
index_name = "applabqatar"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=1536, 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # or your specific region
        )
    )

In [ ]:
response = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key = OPENAI_API_KEY
)

In [ ]:
index = pc.Index(index_name)

In [ ]:
docsearch = PineconeVectorStore(embedding = embedding,index=index)

In [ ]:
uuids = [str(uuid4()) for _ in range(len(text_chunks))]

In [ ]:
docsearch.add_documents(documents=text_chunks, ids=uuids)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
retriever=docsearch.as_retriever(search_kwargs={'k': 10})

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    max_tokens=4096,
    api_key=OPENAI_API_KEY
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
# Example usage
question = "Why does the CDA enforce specific setback requirements for 1 kanal residential plots in Islamabad, and what are these setback regulations?"
response = qa({"query": question})
print(type(response))

# Output response
print("Answer:", response['result'])
print("Source Documents:", response['source_documents'])


In [ ]:
print((response['source_documents'][0]))

In [11]:
pc.delete_index("applabqatar")

In [12]:
index_name = "applabqatar"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=1536, 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # or your specific region
        )
    )